In [13]:
import pandas as pd
import openpyxl
import json
import traceback

In [14]:
class ValidadorCampos:
    """
    Clase para validar campos según reglas extraídas de un archivo Excel.
    
    Contiene reglas de validación para cada campo incluyendo:
    - Nombre del campo (bb)
    - Código del campo (Campo destino)
    - Longitud
    - Tipo (CARACTER, NUMERICO)
    """
    
    def __init__(self, ruta_excel, ruta_json):
        """
        Inicializa el validador cargando las reglas desde un archivo Excel.
        
        Args:
            ruta_excel: Ruta al archivo Excel que contiene las reglas
        """
        self.reglas = {}
        self.ruta_excel = ruta_excel
        self.ruta_json = ruta_json
        self.cargar_reglas_desde_excel()
        self.leer_json()
    
    def cargar_reglas_desde_excel(self):
        try:
            df = pd.read_excel(self.ruta_excel)
            df.columns = df.columns.str.upper()
            
            # Convertir a mayúsculas solo en columnas de texto (CORRECCIÓN)
            for columna in df.select_dtypes(include='object').columns:
                df[columna] = df[columna].str.upper()
            
            # Verificar que las columnas requeridas estén presentes
            columnas_requeridas = ["BB", "CAMPO DESTINO", "TIPO", "LONGITUD"]
            for columna in columnas_requeridas:
                if columna not in df.columns:
                    raise ValueError(f"La columna '{columna}' no existe en el archivo Excel")
            
            # Procesar cada fila y extraer las reglas
            for _, fila in df.iterrows():
                nombre_campo = fila["BB"]
                codigo = fila["CAMPO DESTINO"]
                tipo = fila["TIPO"]
                longitud = float(fila["LONGITUD"])
                
                # Almacenar las reglas en el diccionario
                self.reglas[nombre_campo] = {
                    'codigo': codigo,
                    'tipo': tipo,
                    'longitud': longitud
                    
                }
            
            print(f"Se cargaron {len(self.reglas)} reglas desde el archivo Excel")

            # Imprimir las reglas cargadas
            #print("Reglas cargadas:")
            #for nombre_campo, regla in self.reglas.items():
            #    print(f"{nombre_campo}: {regla}")

        except Exception as e:
            print(f"Error al cargar las reglas desde Excel: {e}")
            raise



    #def leer_json(self):
    #    with open(self.ruta_json, 'r', encoding='utf-8') as f:
    #        datos = json.load(f)
#
    #    # Acceder a "transactions" y obtener los "fields"
    #    transactions = datos.get("body", {}).get("transactions", [])
#
    #    lista_fields = [t["fields"] for t in transactions if "fields" in t]
#
    #    print(lista_fields[0:5])
    #    #return lista_fields  # Retorna una lista con todos los "fields"



    def leer_json(self):
        try:
            with open(self.ruta_json, 'r', encoding='utf-8') as f:
                datos = json.load(f)
                # Acceder a "transactions" y obtener los "fields"
                transactions = datos.get("body", {}).get("transactions", [])
                
                lista_fields = [t["fields"] for t in transactions if "fields" in t]
                print(lista_fields[0:5])
                
                lista_fields_transformados = []
                
                for transaccion in transactions:
                    if "fields" in transaccion:
                        nuevo_fields = {}
                        
                        for campo, valor in transaccion["fields"].items():
                            # Reemplazar el nombre del campo por su código si existe en las reglas
                            codigo = self.reglas.get(campo, {}).get("codigo", campo)
                            nuevo_fields[codigo] = valor
                
                lista_fields_transformados.append(nuevo_fields)
            #return lista_fields_transformados  # Retorna los "fields" con nombres cambiados
            print(lista_fields_transformados[0:5])
    
        except Exception as e:
            print(f"Error al leer o transformar el JSON:\n{traceback.format_exc()}")

In [15]:
ruta_excel = r'C:\Users\ASUS\Documents\Bancolombia\POC_java_vs_python\java_vs_python\Reglas_OPTIMA - copia.xlsx'
ruta_json = r'C:\Users\ASUS\Documents\Bancolombia\POC_java_vs_python\java_vs_python\json_a_homologar.json'

validador = ValidadorCampos(ruta_excel, ruta_json)

Se cargaron 28 reglas desde el archivo Excel
[{'CODIGO TRX': 'AF265', 'DESCRIPCION TRANSACCION': 'DEP EQ ELECTRICOS', 'TIPO CONTABILIDAD': '7115', 'FECHA CONTABLE': '2025-02-19', 'VALOR DB 1': '1000000.02', 'VALOR DB 2': 'N/A', 'VALOR DB 3': 'N/A', 'ID TERCERO DB1': '70', 'ID TERCERO DB2': '71', 'ID TERCERO DB3': 'N/A', 'NOMBRE TERCERO DB1': 'abc', 'NOMBRE TERCERO DB2': 'N/A', 'NOMBRE TERCERO DB3': 'N/A', 'CC DB 1': '123', 'CC DB 2': 'N/A', 'CC DB 3': 'N/A', 'VALOR CR 1': '1000000.02', 'VALOR CR 2': 'N/A', 'VALOR CR 3': 'N/A', 'ID TERCERO CR 1': '80', 'ID TERCERO CR 2': '81', 'ID TERCERO CR 3': 'N/A', 'NOMBRE TERCERO CR1': 'abc', 'NOMBRE TERCERO CR2': 'N/A', 'NOMBRE TERCERO CR3': 'N/A', 'CC CR 1': '123', 'CC CR 2': 'N/A', 'CC CR 3': 'N/A', 'COMENTARIO': 'prueba', 'TIPO DE DOCUMENTO': 'T90', 'NUMERO DE DOCUMENTO': '123', 'DOCUMENTO BASE': 'as', 'NUMERO DOCUMENTO BASE': '123', 'CUENTA': '55604', 'CAMPO_NUMERICO_1': '11420', 'CAMPO_ALFANUMERICO_2': '64507531AA', 'CAMPO_ALFANUMERICO_3': '1